# Validation notebook
Here the FIRES implementation for multiclass and regression is validated and compared to other online features selcetion algorithms.


In [1]:
# import the functions needed fot validate and comparsion

import numpy as np
import pandas as pd
from timeit import default_timer as timer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skmultiflow.data import FileStream
from skmultiflow.neural_networks import PerceptronMask

# using plotly for plots
#import plotly.graph_objects as go
import plotly.express as px



In [2]:
# paint mnist
def paint_digit(digit_values):
    fig = px.imshow(digit_values.reshape(28,28), binary_string=True)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()


In [3]:
# import algorithms
from fires import FIRES
from ofs import OFS, MC_OFS
from ofssgr import OFSSGD, MC_OFSSGD
from fsds import StreamFeatWeight

## Multiclass Data

Here the FIRES softmax implementation is compared to the FSDS, OFS and OFSSGD oun multiclass data.


### Load Datasets as Streaming Data

In [4]:
# MNIST data
stream = FileStream('datasets/Multiclass/mnist_train_normalized.csv', target_idx=0)
stream.prepare_for_use()
dataset_name = "mnist"


### Test Perceptron
the same model is used for all algorithms


In [ ]:
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

### FIRES Framework

In [ ]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax')
print(fires_model.n_mc_samples)

x, y = stream.next_sample(batch_size=100)
# Select features
start_time = timer()
ftr_weights = fires_model.weigh_features(x, y)  # Get feature weights with FIRES
ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
print(timer()-start_time)                    

In [ ]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax')

n_selected_ftr = 100

fires_accuracy = []
fires_f1 = []
fires_times = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample()
    # Select features
    start_time = timer()
    ftr_weights = fires_model.weigh_features(x, y)  # Get feature weights with FIRES
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
    fires_times.append(timer()-start_time)

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_accuracy.append(accuracy_score(y, y_pred))
    fires_f1.append(f1_score(y, y_pred, average=None, labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole fires run took {}".format(fires_run_time))
stream.restart()

In [ ]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


### FSDS algorithm


In [ ]:
stream.restart()
print("Choosen dataset: {}".format(dataset_name))
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

In [ ]:
fsds_model = StreamFeatWeight(m=stream.n_features, k=stream.n_classes)
fsds_model.low_rank_approximation(x.T) # needs some pretraining in the first run

n_selected_ftr = 100

fsds_accuracy = []
#fsds_f1 = []
fsds_times = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)
    # Select features
    start_time = timer()
    ftr_weights = fsds_model.low_rank_approximation(x.T)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
    fsds_times.append(timer()-start_time)

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # Test
    y_pred = predictor.predict(x_reduced)
    
    fsds_accuracy.append(accuracy_score(y, y_pred))
    #fsds_f1.append(f1_score(y, y_pred, average=None, labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fsds_run_time = timer() - start_time_all
print("The whole fsds run took {}".format(fsds_run_time))
stream.restart()

In [ ]:
fig = px.line(y = fsds_accuracy, title="Accuracy for FSDS algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [ ]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


### OFS algorithm

In [ ]:
stream.restart()
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

In [ ]:
n_selected_ftr = 100
ofs = MC_OFS(regularization_param = 0.01, step_size = 0.1, n_selected_ftr=n_selected_ftr, n_total_ftr=stream.n_num_features, n_classes=stream.n_classes)

ofs_accuracy = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)

    # Select features
    for idx, label in enumerate(y):
        ofs.train(x[idx],label)

    selected_ftr = ofs.get_feature_indices()
    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, selected_ftr] = x[:, selected_ftr]

    # Test
    y_pred = predictor.predict(x_reduced)
    ofs_accuracy.append(accuracy_score(y, y_pred))

    # Train
    predictor.partial_fit(x_reduced, y)

end_time_all = timer()
ofs_run_time = timer() - start_time_all
print("The whole ofs run took {}".format(ofs_run_time))

# Restart the FileStream
stream.restart()

In [ ]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


In [ ]:
fig = px.line(y = ofs_accuracy, title="Accuracy for OFS algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

### OFSSGR algorithm

In [5]:
stream.restart()
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [6]:

ofssgd_model = MC_OFSSGD(reduction_threshold=0.4, reduction_value=0.1, regularization_param=0.01, step_size=0.2, n_total_ftrs=stream.n_num_features, n_classes=stream.n_classes)

ofssgd_accuracy = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)

    # Select features
    for idx, label in enumerate(y):
        ofssgd_model.train(x[idx],label)

    selected_ftr = ofssgd_model.get_feature_indices()
    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, selected_ftr] = x[:, selected_ftr]

    # Test
    y_pred = predictor.predict(x_reduced)
    ofssgd_accuracy.append(accuracy_score(y, y_pred))

    # Train
    predictor.partial_fit(x_reduced, y)

end_time_all = timer()
ofssgd_run_time = timer() - start_time_all
print("The whole ofssgd run took {}".format(ofssgd_run_time))

# Restart the FileStream
stream.restart()

The whole ofssgd run took 288.8724846219993


In [7]:
fig = px.line(y = ofssgd_accuracy, title="Accuracy for OFSSGD algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [13]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


In [12]:
selected_ftr

array([ 74,  75,  76, 100, 101, 102, 103, 104, 127, 128, 129, 130, 131,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 234,
       235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
       248, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273,
       274, 275, 276, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298,
       299, 300, 301, 302, 314, 315, 316, 317, 318, 319, 320, 321, 322,
       325, 326, 327, 328, 329, 341, 342, 343, 344, 345, 346, 347, 348,
       349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 369, 370, 371,
       372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384,
       385, 386, 387, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407,
       408, 409, 410, 411, 412, 413, 414, 415, 425, 426, 427, 428, 429,
       432, 433, 434, 435, 436, 437, 438, 441, 442, 443, 453, 45